In [1]:
import pandas as pd
import numpy as np
import random

# File operations
from copy import deepcopy
import glob
import os
from pathlib import Path

# Neuroimaging
import neuromaps
from neuromaps.parcellate import Parcellater
from neuromaps import datasets, images, nulls, resampling, stats
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation, fetch_fslr

# Set seed to 127
random.seed(127)

In [2]:
atlas_data_path = "/taiji1/abry4213/data/neuroimaging_atlases/surfaces/"
fMRI_data_path = "/taiji1/abry4213/data/HCP/S1200/resting_fMRI/"

In [42]:
import nibabel as nib
import numpy as np
import pandas as pd

this_subject = '100307'

# Load the CIFTI grayordinate timeseries
cifti_file = f'{fMRI_data_path}/raw_data/{this_subject}/rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii'
dtseries = nib.load(cifti_file)
data = dtseries.get_fdata()  # shape: (timepoints, grayordinates)

# Load the parcellations (dlabel.nii)
cortical_labels = nib.load(f'{atlas_data_path}/Schaefer/HCP/Yan2023_Homotopic/300Parcels_Yeo2011_7Networks.dlabel.nii').get_fdata().squeeze().astype(int)  # shape: (grayordinates,)
subcortical_labels = nib.load(f'{atlas_data_path}/Tian_Subcortex/Tian_Subcortex_S1_3T_32k.dscalar.nii').get_fdata().squeeze().astype(int)  # shape: (grayordinates,)
dk_cifti_labels = nib.load(f'{atlas_data_path}/DesikanKilliany/fsaverage/atlas-desikankilliany.dscalar.nii').get_fdata().squeeze().astype(int)  # shape: (grayordinates,)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [43]:
# Load lookup tables
cortical_lookup = pd.read_table(f'{atlas_data_path}/Schaefer/HCP/Yan2023_Homotopic/300Parcels_Yeo2011_7Networks_info.txt', header=None)
subcortical_lookup = pd.read_table(f'{atlas_data_path}/Tian_Subcortex/Tian_Subcortex_S1_3T_label.txt', header=None)
dk_lookup = pd.read_csv(f'{atlas_data_path}/DesikanKilliany/fsaverage/dk_lookup_single_hemi.csv')

# Take every other row of cortical_lookup, ignoring the lookup color rows
cortical_lookup = (cortical_lookup
                   .iloc[::2, :]
                   .reset_index(drop=True)
                   .rename(columns={0: 'Region'})
                   .assign(Region_Index = lambda x: range(1, x.shape[0] + 1))
)

subcortical_lookup = (subcortical_lookup
                      .rename(columns={0: 'Region'})
                      .assign(Region_Index = lambda x: range(1, x.shape[0] + 1),
                              Base_Region = lambda x: x['Region'].str.split('-').str[0],
                              Hemisphere = lambda x: x['Region'].str.split('-').str[1])
)

In [5]:
unique_cortical_labels = np.unique(cortical_labels)
unique_cortical_labels = unique_cortical_labels[unique_cortical_labels != 0]  # Exclude unlabeled (0)

# Mean time series for each parcel
Schaefer300_homotopic_parcel_timeseries = []
for label in unique_cortical_labels:
    idx = np.where(cortical_labels == label)[0]
    parcel_ts = data[:, idx].mean(axis=1)  # Mean across grayordinates
    Schaefer300_homotopic_parcel_timeseries.append(parcel_ts)

# Convert to DataFrame for convenience
Schaefer300_homotopic_TS_df = pd.DataFrame(Schaefer300_homotopic_parcel_timeseries).T  # shape: (timepoints, parcels)
Schaefer300_homotopic_TS_df.columns = cortical_lookup.Region


In [6]:
unique_subcortical_labels = np.unique(subcortical_labels)
unique_subcortical_labels = unique_subcortical_labels[unique_subcortical_labels != 0]  # Exclude unlabeled (0)

# Mean time series for each parcel
Tian_S1_subcortex_parcel_timeseries = []
for label in unique_subcortical_labels:
    idx = np.where(subcortical_labels == label)[0]
    parcel_ts = data[:, idx].mean(axis=1)  # Mean across grayordinates
    Tian_S1_subcortex_parcel_timeseries.append(parcel_ts)

# Convert to DataFrame for convenience
Tian_S1_subcortex_TS_df = pd.DataFrame(Tian_S1_subcortex_parcel_timeseries).T  # shape: (timepoints, parcels)
Tian_S1_subcortex_TS_df.columns = subcortical_lookup.Region

In [44]:
unique_DK_labels = np.unique(dk_cifti_labels)
unique_DK_labels = unique_DK_labels[unique_DK_labels != 0]  # Exclude unlabeled (0)

# Mean time series for each parcel
DesikanKilliany_parcel_timeseries = []
for label in unique_DK_labels:
    idx = np.where(dk_cifti_labels == label)[0]
    parcel_ts = data[:, idx].mean(axis=1)  # Mean across grayordinates
    DesikanKilliany_parcel_timeseries.append(parcel_ts)

# Convert to DataFrame for convenience
DesikanKilliany_TS_df = pd.DataFrame(DesikanKilliany_parcel_timeseries).T  # shape: (timepoints, parcels)
DesikanKilliany_TS_df.columns = dk_lookup['Region'].values